In [ ]:
# Connor Murray
# Measure expression changes for specific genes
# Load packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Define file paths
gene_counts_file = "../output/rna/norm_tmm.tsv"  # Path to the TMM normalized gene counts
metadata_file = "/standard/vol185/cphg_Manichaikul/users/csm6hg/metadata/metadata_10_17_2024_CSM.txt"  # Path to your metadata file
coloc_file = "../output/coloc/coloc_eqtl_candidates_full.txt"

# Read in the gene counts data
gene_counts = pd.read_csv(gene_counts_file, sep='\t', index_col=0) # index_col=0 uses the first column as the index

# Read in the metadata
metadata = pd.read_csv(metadata_file, sep='\t')

# Read in coloc
coloc = pd.read_csv(coloc_file, sep='\t')
coloc = coloc[coloc['PP.H4'] > 0.9][['gene', 'gene.y', 'common_gene']].rename(columns={'gene':'gene_simp', 'gene.y':'gene'}) # Extract best genes
coloc = coloc[~coloc.duplicated()]
#coloc = coloc[coloc["min_p.eqtl"] < 1e-10]["gene"]
#print(coloc.head())

# Display the first few rows of each dataframe to verify
print("Gene Counts Data:", gene_counts.shape)
#print(gene_counts.head())
print("\nMetadata:", metadata.shape)
#print(metadata.head())
print("\nColoc", coloc.shape)
print(coloc)

In [ ]:
# Transpose gene_counts so sample IDs are in the index
gene_counts_transposed = gene_counts.transpose()

# Reset index of gene_counts_transposed to make sample IDs a column
gene_counts_transposed = gene_counts_transposed.reset_index(names = ['gene'])
#print(gene_counts_transposed)

# Rename the index column to 'SAMPLE_ID_TOR' to match the metadata
gene_counts_transposed = gene_counts_transposed.rename(columns={'gene': 'SAMPLE_ID_TOR'})

# Merge the two dataframes using 'SAMPLE_ID_TOR'
merged_data = pd.merge(gene_counts_transposed, metadata, on='SAMPLE_ID_TOR', how='inner')
merged_data = merged_data[merged_data['Age_at_collection'].notna()]
#print(merged_data["diagnosis_simple"].value_counts())
merged_data = merged_data[merged_data['diagnosis_simple'].isin(["Non-Failing", "ICM", "IDCM"])]

# Display the first few rows of the merged dataframe
print("Merged Data:", merged_data.shape)
print(merged_data.head())

In [ ]:
# Restrict available gene columns to those present in merged_data
available_gene_cols = [col for col in coloc['gene'] if col in merged_data.columns]
print(f"Number of gene columns in merged_data that are in coloc: {len(available_gene_cols)}")

# Define your metadata columns
metadata_cols = ['SAMPLE_ID_TOR', 'diagnosis_simple', 'Age_at_collection',
    'Collection_year', 'Biosample_affection_status', 'Analyte_isolation_lab',
    'ANALYTE_TYPE', 'Sample_container_ID', 'Sample_well_ID', 'Assay_lab',
    'Mapping_Rate', 'Total_Reads']

# Create a new DataFrame with only the metadata columns and the available gene columns
re_data = merged_data[metadata_cols + available_gene_cols].copy()

# Create a mapping dictionary from long gene names (coloc['gene']) to common gene names (coloc['common_gene'])
mapping = dict(zip(coloc['gene'], coloc['common_gene']))

# Only columns that match keys in mapping will be renamed.
re_data.rename(columns=mapping, inplace=True)

# Display the updated DataFrame
print(re_data.columns)

In [ ]:
# Plotting the distribution of a specific gene
if available_gene_cols:
    gene_to_plot = "FLNC"
    plt.figure(figsize=(10, 6))
    sns.histplot(re_data[gene_to_plot], kde=True)
    plt.title(f'Distribution of {gene_to_plot}')
    plt.xlabel('Normalized Gene Counts')
    plt.ylabel('Frequency')
    plt.show()

    # Scatter plot of gene expression vs. a metadata variable (e.g., age)
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x='Age_at_collection', 
                    y=gene_to_plot, 
                    hue="diagnosis_simple",
                    data=re_data)
    plt.title(f'{gene_to_plot} vs. Age')
    plt.xlabel('Age')
    plt.ylabel('Normalized Gene Counts')
    plt.show()

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.patches as patches

if available_gene_cols:
    gene_to_plot = "FLNC"
    
    # 1. Plot the histogram with KDE
    plt.figure(figsize=(10, 6))
    sns.histplot(re_data[gene_to_plot], kde=True)
    plt.title(f'Distribution of {gene_to_plot}')
    plt.xlabel('Normalized Gene Counts')
    plt.ylabel('Frequency')
    plt.show()

    # 2. Plot the scatter plot colored by diagnosis group
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x='Age_at_collection', 
                    y=gene_to_plot, 
                    hue="diagnosis_simple",
                    data=re_data)
    plt.title(f'{gene_to_plot} vs. Age')
    plt.xlabel('Age at Collection')
    plt.ylabel('Normalized Gene Counts')
    plt.show()

    # 3. Perform clustering within each diagnosis group (using 2 clusters per group as an example)
    cluster_data = re_data.copy()
    cluster_labels = pd.Series(index=cluster_data.index, dtype="object")
    
    for group in cluster_data['diagnosis_simple'].unique():
        group_mask = cluster_data['diagnosis_simple'] == group
        group_data = cluster_data.loc[group_mask, ['Age_at_collection', gene_to_plot]]
        
        if len(group_data) >= 2:
            kmeans = KMeans(n_clusters=2, random_state=42)
            labels = kmeans.fit_predict(group_data)
            cluster_labels.loc[group_mask] = [f"{group}_cluster_{lbl}" for lbl in labels]
        else:
            cluster_labels.loc[group_mask] = f"{group}_cluster_0"
    
    cluster_data['cluster'] = cluster_labels

    # 4. Create a scatter plot with clusters and add circles around each cluster group.
    plt.figure(figsize=(10, 6))
    ax = sns.scatterplot(
        x='Age_at_collection', 
        y=gene_to_plot, 
        hue='cluster', 
        style='diagnosis_simple',  # Keep original diagnosis markers
        data=cluster_data,
        palette="tab10"
    )
    plt.title(f'{gene_to_plot} vs. Age with Cluster Circles')
    plt.xlabel('Age at Collection')
    plt.ylabel('Normalized Gene Counts')

    # Build a color dictionary for clusters (using the same palette as the scatter plot)
    unique_clusters = cluster_data['cluster'].unique()
    palette = sns.color_palette("tab10", len(unique_clusters))
    color_dict = dict(zip(unique_clusters, palette))

    # Add circles around each cluster: compute centroid and max distance from centroid as radius
    for cluster_label in unique_clusters:
        cluster_points = cluster_data[cluster_data['cluster'] == cluster_label]
        if cluster_points.empty:
            continue
        center_x = cluster_points['Age_at_collection'].mean()
        center_y = cluster_points[gene_to_plot].mean()
        # Calculate distances of points from the centroid
        distances = np.sqrt((cluster_points['Age_at_collection'] - center_x)**2 + 
                            (cluster_points[gene_to_plot] - center_y)**2)
        # Set the radius a bit larger than the max distance to enclose all points
        radius = distances.max() * 1.1  
        circle = plt.Circle((center_x, center_y), radius, 
                            color=color_dict[cluster_label], 
                            fill=False, lw=2, alpha=0.7)
        ax.add_patch(circle)

    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()

In [ ]:
# Define your metadata columns
metadata_cols = ['SAMPLE_ID_TOR', 'diagnosis_simple', 'Age_at_collection']

# available_gene_cols is assumed to be a list of columns that are gene expression values
value_vars = [gene for gene in coloc['common_gene'].tolist() if gene in re_data.columns]

# Melt the DataFrame:
m_data = pd.melt(
    re_data,
    id_vars=metadata_cols,
    value_vars=value_vars,
    var_name='gene',
    value_name='normalized_count')

# Display the first few rows of the melted DataFrame
print(m_data.head())

# Boxplot per gene
plt.figure(figsize=(12, 8))
sns.boxplot(data=m_data, 
            y='gene', 
            x='normalized_count', 
            hue='diagnosis_simple',
            showfliers=False)  # Optionally, hide outliers for clarity
plt.title("Distribution of Normalized Counts by Gene and Disease State (Boxplot)")
plt.xlabel("Normalized Counts")
plt.ylabel("Gene")
plt.legend(title='Diagnosis', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import math
from scipy import stats

# Step 1: Create a list of gene columns using the common gene names that are present in re_data.
value_vars = [gene for gene in coloc['common_gene'].tolist() if gene in re_data.columns]
print("Genes to analyze:", len(value_vars))

# Step 2: Define a function to compute Cohen's d and perform a t-test.
def compute_effect_size_and_ttest(x, y):
    """
    Calculate Cohen's d for two arrays of values and perform a t-test.
    
    Parameters:
        x: Values from the control group.
        y: Values from the case group.
        
    Returns:
        d: Cohen's d effect size.
        t: t-test statistic.
        p: p-value from the t-test.
    """
    x = np.array(x)
    y = np.array(y)
    n1, n2 = len(x), len(y)
    mean1, mean2 = np.mean(x), np.mean(y)
    # Use sample standard deviation (ddof=1)
    s1, s2 = np.std(x, ddof=1), np.std(y, ddof=1)
    # Calculate pooled standard deviation
    pooled_std = np.sqrt(((n1 - 1) * s1**2 + (n2 - 1) * s2**2) / (n1 + n2 - 2))
    
    # Avoid division by zero for effect size calculation
    if pooled_std == 0:
        d = np.nan
    else:
        d = (mean2 - mean1) / pooled_std
        
    # Perform t-test (assuming equal variances)
    t, p = stats.ttest_ind(x, y, equal_var=True)
    return d, t, p

# Step 3: Compute effect sizes, p-values, and sample sizes.
# Here, we assume "Non-Failing" is the control group.
controls = re_data[re_data['diagnosis_simple'] == "Non-Failing"]

# Prepare a list to collect the results.
effect_sizes = []

# Identify all case groups (i.e., groups that are not "Non-Failing")
case_groups = re_data[re_data['diagnosis_simple'] != "Non-Failing"]['diagnosis_simple'].unique()

# For each gene, calculate Cohen's d and t-test comparing controls to each case group.
for gene in value_vars:
    for case in case_groups:
        case_group = re_data[re_data['diagnosis_simple'] == case]
        # Drop any NaN values for the gene of interest before calculating.
        control_values = controls[gene].dropna()
        case_values = case_group[gene].dropna()
        n_controls = len(control_values)
        n_cases = len(case_values)
        if n_controls > 1 and n_cases > 1:
            d, t_stat, p_value = compute_effect_size_and_ttest(control_values, case_values)
        else:
            d, t_stat, p_value = np.nan, np.nan, np.nan
        effect_sizes.append({
            'gene': gene,
            'case_group': case,
            'effect_size': d,
            't_statistic': t_stat,
            'p_value': p_value,
            'n_controls': n_controls,
            'n_cases': n_cases
        })

# Convert the results into a DataFrame for easier viewing.
effect_sizes_df = pd.DataFrame(effect_sizes)
print(effect_sizes_df.sort_values('effect_size', ascending=False))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from adjustText import adjust_text

# Use a clean white grid style
sns.set_theme(style="whitegrid")

# 1. Sort the DataFrame in ascending order by effect_size within each case_group
effect_sizes_df = effect_sizes_df.sort_values(by='effect_size', ascending=True)
effect_sizes_df['x'] = effect_sizes_df.groupby('case_group').cumcount()
epsilon = 1e-10
effect_sizes_df['p_value_log'] = -np.log10(effect_sizes_df['p_value'] + epsilon)
g = sns.FacetGrid(effect_sizes_df, col='case_group', col_wrap=2, height=5, aspect=1.2, sharex=False)

# Map the scatterplot to each facet, using the p_value_log for color.
g.map_dataframe(
    sns.scatterplot,
    x='x',
    y='effect_size',
    hue='p_value_log',
    palette='viridis',
    s=100,
    edgecolor='black')

# For each facet, add a horizontal line at y=0 and enable gridlines.
for ax in g.axes.flat:
    ax.axhline(0, color='black', linestyle='--', linewidth=1)
    ax.grid(True, linestyle='--', linewidth=0.5, color='gray', alpha=0.5)

# Add a Colorbar on the Left Side
norm = Normalize(vmin=effect_sizes_df['p_value_log'].min(), vmax=effect_sizes_df['p_value_log'].max())
sm = ScalarMappable(norm=norm, cmap='viridis')
sm.set_array([])

# Create a new axis for the colorbar on the left (adjust the coordinates as needed)
cax = g.fig.add_axes([0.07, 0.15, 0.02, 0.7])  # [left, bottom, width, height]
cbar = g.fig.colorbar(sm, cax=cax, orientation='vertical', label='-log10(p-value)')

# Adjust the subplots so that the left margin accommodates the colorbar.
g.fig.subplots_adjust(left=0.15)
g.set_titles("{col_name}", fontweight='bold')

# Annotate Each Point with Gene Names (Italicized) and Arrows
for ax in g.axes.flat:
    # Get the case group from the facet title (which is the group label)
    group = ax.get_title()
    subset = effect_sizes_df[effect_sizes_df['case_group'] == group]
    texts = []
    
    for _, row in subset.iterrows():
        # Add text annotation with an italic font style
        txt = ax.text(row['x'] + 0.2, row['effect_size'] + 0.02, row['gene'],
                      fontsize=9, fontstyle='italic', color='darkblue')
        texts.append(txt)
        # Draw an arrow from the point to the annotation
        ax.annotate(
            '', 
            xy=(row['x'], row['effect_size']),
            xytext=(row['x'] + 0.2, row['effect_size'] + 0.02),
            arrowprops=dict(arrowstyle='->', color='gray', lw=0.5)
        )
    # Adjust text positions to reduce overlapping
    adjust_text(texts, ax=ax, arrowprops=dict(arrowstyle='->', color='gray', lw=0.5))

plt.tight_layout(rect=[0.15, 0, 1, 1])  # leave space on the left for the colorbar
plt.show()